Task: Predict author gender based on Blog Authorship Corpus

Columns description:  

* gender   = male or female - you have to predict that for test dataset
* text  = blogpost text

Most of the reviews are in English, but there are a few in other languages.

Metric: accuracy  
baseline: 0.6423

In [1]:
import pandas as pd
import numpy as np

In [10]:
df_train = pd.read_csv('train.csv')
df_train.head()

,Unnamed: 0,gender,text
0,0,male,I'm going to Glastonbury.
1,1,male,"Lets see, what did I do on Monday..."
2,2,female,Listening to music from the 60s and 70s...
3,3,female,"8:26 a.m. Sunday, Day 97. Daylight ..."
4,4,female,'Do you want me to tell you s...


In [16]:
df_test = pd.read_csv('test.csv')
df_test.head()

,Unnamed: 0,gender,text
0,0,female,"ok, comments are fixed."
1,1,female,Just remembered.......now i've stopped ...
2,2,male,"To take this test, kindly visit urlLin..."
3,3,male,Suggested topics for Bloggs 1) researc...
4,4,male,Foo. Yesterday was quite scary. We had...


Попробуем превзойти бейзлайн с помощью наивного байеса:

In [22]:
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB

vec = TfidfVectorizer(lowercase=True, analyzer='word', ngram_range=(1, 1), norm=None, use_idf=False, binary=True)
X = vec.fit_transform(df_train.text)
print('feature matrix shape', X.shape)

# encode class labels
label_enc = LabelEncoder().fit(df_train.gender)
y_train = label_enc.transform(df_train.gender)

# fit our prediction model
model = MultinomialNB(alpha=1.0)
model.fit(X, y_train)
print('train', metrics.accuracy_score(y_train, model.predict(X)))

# perfomance on test dataset
X_test = vec.transform(df_test.text)
y_pred = model.predict(X_test)
y_test = label_enc.transform(df_test.gender)
print('test', metrics.accuracy_score(y_test, model.predict(X_test)))

feature matrix shape (10900, 67074)
train 0.8342201834862385
test 0.6309611151870873


Наивный байес не превзошел бейзлайн, попробуем логистическую регрессию:

In [26]:
from sklearn.linear_model import LogisticRegression

SEED = 1337
vec = TfidfVectorizer(lowercase=True, analyzer='char', ngram_range=(3, 3), norm='l2', use_idf=True, smooth_idf=True)
X = vec.fit_transform(df_train.text)
print('feature matrix shape', X.shape)

#encode labels
label_enc = LabelEncoder().fit(df_train.gender)
y_train = label_enc.transform(df_train.gender)

model = LogisticRegression(random_state=SEED)
model.fit(X, y_train)
print('train', metrics.accuracy_score(y_train, model.predict(X)))

# perfomance on test dataset
X_test = vec.transform(df_test.text)
y_pred = model.predict(X_test)
y_test = label_enc.transform(df_test.gender)
print('test', metrics.accuracy_score(y_test, model.predict(X_test)))

feature matrix shape (10900, 45291)
train 0.7722018348623854
test 0.644534115920763
